In [ ]:
# helper functions

import re

# replaces 'region *', 'vhs', and 'dvd' (case insensitive) in product names
def name_transformation(name):
    pattern = re.compile(r'(?i)region .|dvd|vhs')
    name = re.sub(pattern, '', name)
        
    return name

# converts a month to a number for date formatting
def month_to_number(month):
    conversions = {'January':'01',
            'February':'02', 
            'March':'03',
            'April':'04', 
            'May':'05', 
            'June':'06',
            'July':'07',
            'August':'08',
            'September':'09',
            'October':'10',
            'November':'11',
            'December':'12'}
    if month in conversions:
        return conversions[month]
    else:
        return -1

In [ ]:
import os
import requests
from bs4 import BeautifulSoup as soup
max_product_name = 0

def get_reviews(page):
    reviews = []   
    productID = product_id(page)
    productName = product_name(page)
    
    # get all the reviews from one file
    reviews_html = page.find_all('div', {'data-hook': 'review'})
    for review_html in reviews_html:
        reviewBody = review_body(review_html)
        
        # empty body, no index this review
        if(reviewBody == None):
            continue
        
        review_json = {'productID' : productID,
                'productName' : productName,
                'reviewSummary': review_summary(review_html),
                'reviewBody': reviewBody,
                'reviewDate': review_date(review_html),
                'reviewScore': review_score(review_html)}
        reviews.append(review_json)
    return reviews

def product_id(page):
    productID = page.find('input', {'id': 'ASIN'})
    if(productID == None):
        # prime video format
        video = page.find('div', {'data-automation-id': 'watchlist-button'})
        productID = video.find('input', {'name': 'itemId'})
    return productID['value']

def product_name(page):
    global max_product_name
    productName = page.find('span', {'id': 'productTitle'})
    if(productName == None):
        # prime video format
        productName = page.find('h1', {'data-automation-id': 'title'})
    productName = name_transformation(productName.text).strip()
    # for query purpose
    if(len(productName) > max_product_name):
        max_product_name = len(productName)
    return productName

def review_summary(review):
    return review.find('a', {'data-hook': 'review-title'}).span.text

def review_body(review):
    body = review.find('span', {'data-hook': 'review-body'})
    if(body == None):
        return None
    return body.span.text

def review_date(review):
    date = review.find('span', {'data-hook': 'review-date'}).text
    date = date.split()
    
    month = month_to_number(date[0])
    
    day = date[1][:-1]
    if len(day) == 1:
        day = '0' + day
        
    year = date[2]
  
    return year + '-' + month + '-' + day + 'T00:00:00Z'

def review_score(review):
    score = review.find('i', {'data-hook': 'review-star-rating'}).span.text
    return int(score[0:1])

def read_file(filename):
    with open(filename, encoding='utf8') as f:
        return f.read()

def scrape_pages_for_reviews(path):
    reviews = []
    for filename in os.listdir(path):
        file_text = read_file(f'{path}/{filename}')
        page = soup(file_text, 'html5lib')
        reviews += get_reviews(page)
    return reviews

In [ ]:
reviews_json = scrape_pages_for_reviews('pages')

In [ ]:
import subprocess

SOLR_EXECUTABLE = 'C:\\solr-8.0.0\\bin\\solr.cmd'

def solr_command(*args):
    return subprocess.check_output([SOLR_EXECUTABLE] + list(args))

In [ ]:
import pysolr

def create_collection(config_dir, collection_name = 'amzn-reviews'):
    solr_command('create_collection', '-c', collection_name, '-d', config_dir)
    
    return None

In [ ]:
create_collection('reviews')

In [ ]:
def index_reviews(reviews, solr_port=8983, collection_name='amzn-reviews'):
    solr = pysolr.Solr(f'http://localhost:{solr_port}/solr/{collection_name}')
    solr.add(reviews, commit=True)
    return None

In [ ]:
index_reviews(reviews_json)

In [ ]:
'''
Review query. returns the following fields:

    productName
    productID
    reviewSummary
    reviewDate
    reviewScore
    
'''

from dateutil.parser import parse

# return whether the string can be interpreted as a date
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False


def review_query_dictionary(kwd_str='', date_str='', score_str=''):
    dict = {'fields':'productName AND productID AND reviewSummary AND reviewDate AND reviewScore'}
    
    # adds keywords to query, if any
    if kwd_str != '':
        dict['_review_'] = kwd_str
    
    # checks formatting of date string and adds to query, if not blank
    date_list = date_str.split()

    if len(date_list) != 0:
        if len(date_list) == 2 and is_date(date_list[1]):
            date = date_list[1] + 'T00:00:00Z'
        else:
            raise ValueError('ERROR: Invalid review date formatting.')
            return None
    
        if date_list[0] == '>=':
            dict['reviewDate'] = '[' + date + ' TO *]'
        elif date_list[0] == '<=':
            dict['reviewDate'] = '[* TO ' + date + ']'
        else:
            raise ValueError('ERROR: Invalid review date formatting.')
            return None
    
    # checks formatting of score string and adds to query, if not blank
    score_list = score_str.split()

    if len(score_list) != 0:
        if len(score_list) == 2 and score_list[1].isdigit():
            score = score_list[1]
        else:
            raise ValueError('ERROR: Invalid review score formatting.')
            return None

        if score_list[0] == '>=':
            dict['reviewScore'] = '[' + score + ' TO *]'
        elif score_list[0] == '<=':
            dict['reviewScore'] = '[* TO ' + score + ']'
        else:
            raise ValueError('ERROR: Invalid review score formatting.')
            return None

    return dict


In [ ]:
'''
ASIN query. returns the following fields:
 
    reviewSummary
    reviewBody
    reviewDate
    reviewScore
   
'''

def asin_query_dictionary(asin):
    return {'fields':'reviewSummary AND reviewBody AND reviewDate AND reviewScore', 'productID':asin}

In [ ]:
'''
name query. returns the following fields:
 
    productName
    productID
    reviewSummary
    reviewDate
    reviewScore
    id
 
'''

def product_name_query_dictionary(name_terms):
    name_query = {}
    # query order matching
    name_query['complexphrase'] = '{!complexphrase inOrder=true}'
    name_query['productName'] = f'\"{name_terms}\"~{max_product_name}'
    name_query['fields'] = 'productName AND productID and reviewSummary AND reviewDate AND reviewScore AND id'
    
    return name_query

In [ ]:
'''
id query. returns the following fields:
 
    productName
    productID
    reviewSummary
    reviewDate
    reviewScore
    id
 
'''

def id_query_dictionary(id):
    id_query = {}
    id_query['id'] = id
    id_query['fields'] = 'productName AND productID AND reviewSummary AND reviewDate AND reviewDate AND reviewScore AND id'
    
    return id_query

In [29]:
import requests

def get_query(params):
    query = 'q='
    fields = ''
    if('fields' in params):
        fields = '&fl=' + params['fields']
        
    if('complexphrase' in params):
        query += params['complexphrase']
    
    for key in params.keys():
        if(key != 'fields' and key != 'complexphrase'):
            query += key + ':' + params[key] + ' AND '
    # removing last 'AND'
    query = query[:len(query) - 5]
    
    return query + fields

def do_query(params, port='8983', collection='amzn-reviews'):
    if params is None:
        raise TypeError('ERROR: Empty params.')
        return None
    
    url = f'http://localhost:{port}/solr/{collection}/select'
    request_string = url + '?' + get_query(params)
    response = requests.get(request_string)
    
    if(response.status_code != 200):
        raise requests.exceptions.HTTPError('HTTP ERROR: status code ' + str(response.status_code))
        return None
          
    return response.json()['response']['docs']